In [1]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install evaluate
!pip install -U datasets==2.20.0 pyarrow==15.0.2 transformers==4.44.2 evaluate==0.4.2 --no-cache-dir
!pip install -q datasets evaluate accelerate scikit-learn pandas matplotlib
!pip install -U "transformers>=4.41" accelerate safetensors
!pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 42.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.
   ━━━━━━━━

In [2]:
!pip install --upgrade --no-cache-dir git+https://github.com/Shannu3766/bi_influence.git

  Cloning https://github.com/Shannu3766/bi_influence.git to /tmp/pip-req-build-3iypip4d
  Running command git clone --filter=blob:none --quiet https://github.com/Shannu3766/bi_influence.git /tmp/pip-req-build-3iypip4d
  Resolved https://github.com/Shannu3766/bi_influence.git to commit 981f2a3a9b906864b4f65765cb42335892489a46
  Preparing metadata (setup.py) ... done
  Created wheel for adaptive_lora: filename=adaptive_lora-2.1.0-py3-none-any.whl size=5843 sha256=fab769e7eb52f37fd94740c8d5617e6d565f31e8fecc76c8c4cde207b10a15e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-_do9v0y7/wheels/4b/91/1d/e153ff3aa3759d6ccdcc4706d8559f1cbfaa4aa22b992958cd
Successfully built adaptive_lora


In [3]:
import os
from typing import Optional

import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, TaskType
import evaluate
from adaptive_lora.callbacks import AdaptiveLoRACallback

2025-12-03 10:17:40.785761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764757061.008296      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764757061.071200      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [4]:
model_checkpoint = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" 
output_dir = "./tinyllama-qnli-lora"
seed = 42
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [5]:
batch_size = 16
eval_batch_size = 32
num_train_epochs = 3
max_length = 128
learning_rate = 2e-5
weight_decay = 0.01
rank=8
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=rank,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=[   
        "q_proj", "k_proj", "v_proj", "o_proj", 
        "gate_proj", "up_proj", "down_proj"
    ],)

In [6]:

def clean_text(s: Optional[str]) -> str:
    if s is None:
        return ""
    return " ".join(str(s).strip().split())

def build_one_shot_demo(example: dict) -> str:
    q = clean_text(example["question"])
    c = clean_text(example["context"])
    lbl = example.get("label_text", "Yes")
    return f"Example:\nQuestion: {q}\nContext: {c}\nAnswer (Yes/No): {lbl}\n\n"

In [7]:
ONE_SHOT_EXAMPLE = {
    "question": "Who wrote Hamlet?",
    "context": "Hamlet was written by William Shakespeare and first performed in the early 17th century.",
    "label_text": "Yes",  # "Yes" => the context contains the answer
}
from datasets import load_dataset
import time

def load_dataset_with_retry(path, name=None, max_retries=None, wait=2):
    """
    Repeatedly tries to load a dataset until it succeeds.
    - max_retries: None → infinite retries
    - wait: base wait time (exponential backoff)
    """
    attempt = 0

    while True:
        try:
            if name is not None:
                ds = load_dataset(path, name)
            else:
                ds = load_dataset(path)

            print(f"Dataset loaded successfully after {attempt} attempts.")
            return ds

        except Exception as e:
            attempt += 1
            wait_time = wait * min(5, attempt)  # exponential cap

            print(f"[Attempt {attempt}] Failed to load dataset: {e}")
            print(f"Retrying in {wait_time} seconds...\n")

            # If user gives a max_retries limit
            if max_retries is not None and attempt >= max_retries:
                print("Max retries reached. Raising error.")
                raise e

            time.sleep(wait_time)
dataset = load_dataset_with_retry("glue", "qnli")

[Attempt 1] Failed to load dataset: 429 Client Error: Too Many Requests for url: https://us.gcp.cdn.hf.co/xet-bridge-us/621ffdd236468d709f181e3f/31889d28d37df009587823b2c97e9d7ab4bcdbb8ad3a51e33bd6c31cc2041c05?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27validation-00000-of-00001.parquet%3B+filename%3D%22validation-00000-of-00001.parquet%22%3B&Expires=1764760684&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY0NzYwNjg0fX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjIxZmZkZDIzNjQ2OGQ3MDlmMTgxZTNmLzMxODg5ZDI4ZDM3ZGYwMDk1ODc4MjNiMmM5N2U5ZDdhYjRiY2RiYjhhZDNhNTFlMzNiZDZjMzFjYzIwNDFjMDVcXD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=oNneJqPLmcxJT2JtOTPWRoqtV-bSwRZ7W9z93DhHGdMlLtn7icY3y2mG9dkkSLngLsZdckVHcSTJzX9Bi4BrNFzQUYYEH1VAL5~-h9hKdZUAczo9IDZWrAFTIjefzd-0O54GBI6pp8ri8FV4Axqz4~aF-XMYbu51G9uSb6hrG8qI0vWlMKT0QhDRkYIDbKqShP6I~SvOtN1YizfaytXW0gLSafJ~z2CD0OE0V5P2jmOFLknV3-UZzUHsZANPdEr8rdTU~kLszYJChsXS

[Attempt 2] Failed to load dataset: 429 Client Error: Too Many Requests for url: https://us.gcp.cdn.hf.co/xet-bridge-us/621ffdd236468d709f181e3f/cb1efbbeb399c9332adba8921db9249d19990a0514890c55641cc39be30b9b0c?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27test-00000-of-00001.parquet%3B+filename%3D%22test-00000-of-00001.parquet%22%3B&Expires=1764760688&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY0NzYwNjg4fX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjIxZmZkZDIzNjQ2OGQ3MDlmMTgxZTNmL2NiMWVmYmJlYjM5OWM5MzMyYWRiYTg5MjFkYjkyNDlkMTk5OTBhMDUxNDg5MGM1NTY0MWNjMzliZTMwYjliMGNcXD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=dzk1sCJpnhcW4lMBlwA90~WinarpFOei8BoDJxNviTvL-E79x0CyC47-WqADUVtW6N2ahNU2ibQ6pqnMh299XcCveGKAw4TojgdXP1l2FZa-R~2VTbpvxsVzLs9dszzsj9dXZeuX6xiu~7gCwLgTrp0QdOqJxsmG42J2151hnev~LQT3DH547BTrPgQItJFMkyAJEnpfgvMNpspkQ~4xC3-Ta9yS1yCMsWGBydQDUIOGBWz4RRhZ-yruXHftchMOmEuOp9EiKcYrvF8UF3-mvEKFGya4

Generating train split:   0%|          | 0/104743 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Dataset loaded successfully after 2 attempts.


In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

if tokenizer.pad_token is None:
    # safe default: use eos_token as pad
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [9]:

# Load tokenizer


def preprocess_function(examples, *, max_length: int = max_length, add_demo: bool = False):
    """
    Build instruction-like prompts and tokenize them.
    - add_demo: if True, prepends a 1-shot example to each prompt (can help decoder-only models)
    """
    questions = [clean_text(q) for q in examples["question"]]
    contexts = [clean_text(s) for s in examples["sentence"]]

    demo_str = build_one_shot_demo(ONE_SHOT_EXAMPLE) if add_demo else ""

    prompts = []
    for q, c in zip(questions, contexts):
        # Instruction-style prompt ending with a short, constrained label target
        prompt = (
            "You are a helpful assistant.\n"
            f"{demo_str}"
            f"Question: {q}\n"
            f"Context: {c}\n"
            "Answer (Yes/No):"
        )
        prompts.append(prompt)

    tokenized = tokenizer(
        prompts,
        truncation=True,
        max_length=max_length,
        padding=False, 
    )

    if "label" in examples:
        tokenized["labels"] = examples["label"]
    elif "labels" in examples:
        tokenized["labels"] = examples["labels"]

    return tokenized

print("Tokenizing dataset...")
tokenized = dataset.map(
    lambda ex: preprocess_function(ex, max_length=max_length, add_demo=False),
    batched=True,
    remove_columns=["question", "sentence", "idx"],
)

if "label" in tokenized["train"].column_names and "labels" not in tokenized["train"].column_names:
    tokenized = tokenized.rename_column("label", "labels")

tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest", pad_to_multiple_of=8)
train_dataset = tokenized["train"].select(range(10000))
eval_dataset = tokenized["validation"].select(range(1500))

Tokenizing dataset...


Map:   0%|          | 0/104743 [00:00<?, ? examples/s]

Map:   0%|          | 0/5463 [00:00<?, ? examples/s]

Map:   0%|          | 0/5463 [00:00<?, ? examples/s]

In [10]:
print("Loading model...")
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2,
    device_map="auto",
    torch_dtype=torch.bfloat16, 
    trust_remote_code=True
)
print("model Loaded")

Loading model...


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model Loaded


In [11]:

# Resize token embeddings if tokenizer changed
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

# -------------------------
# Apply LoRA (PEFT)
# -------------------------
print("Applying LoRA (PEFT)...")
model = get_peft_model(model, peft_config)

Applying LoRA (PEFT)...


In [12]:
val_dataloader = torch.utils.data.DataLoader(
    eval_dataset,
    batch_size=eval_batch_size,
    shuffle=False,
    collate_fn=data_collator,
    pin_memory=torch.cuda.is_available(),
    num_workers=2,
)

In [13]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    acc = accuracy.compute(predictions=preds, references=labels)
    return {"accuracy": acc["accuracy"]}

# -------------------------
# TrainingArguments + Trainer
# -------------------------
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    num_train_epochs=num_train_epochs,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    seed=seed,
    fp16=False,
    bf16=True,
    push_to_hub=False,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Starting training...")
trainer.train()
print("Saving model and adapters...")
trainer.save_model(output_dir)
print("Saved model to", output_dir)

/tmp/ipykernel_20/1032849091.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting training...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.388100,0.368996,0.795333
2,0.334900,0.350768,0.806000
3,0.268100,0.342761,0.818667


Saving model and adapters...
Saved model to ./tinyllama-qnli-lora


In [14]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.3427613079547882, 'eval_accuracy': 0.8186666666666667, 'eval_runtime': 203.0991, 'eval_samples_per_second': 7.386, 'eval_steps_per_second': 0.231, 'epoch': 3.0}
